
# Data lakes y Teradata



## Qué es un data lake?


Un data lake es un repositorio centralizado que permite almacenar datos en su forma más cruda:
- Estructurados, como archivos CSV
- Semiestructurados, como JSON y XML
- No estructurados, como videos, logs, imágenes y PDFs

Características clave:
- No necesita un esquema rígido al ingresar los datos (schema-on-read)
- Ideal para Big Data y Machine Learning
- Permite almacenar grandes volúmenes a bajo costo

Ejemplos de tecnologías:
- Azure Data Lake
- Amazon S3
- Hadoop Distributed File System (HDFS)


## Diferencias clave entre base de datos y data lake


Base de datos:
- Tipo de datos: Estructurados
- Esquema: Fijo (schema-on-write)
- Velocidad de consulta: Alta para datos bien indexados
- Casos de uso: Operacionales, reportes BI
- Tecnologías de ejemplo: RDBMS, SQL

Data Lake:
- Tipo de datos: Estructurados, semiestructurados y no estructurados
- Esquema: Flexible (schema-on-read)
- Velocidad de consulta: Lenta si no se preparan los datos
- Casos de uso: Análisis exploratorio, ciencia de datos, ML
- Tecnologías de ejemplo: Hadoop, Spark, Cloud Storage


## Por qué se usa ELT con data lakes?


- Cargar sin transformar permite velocidad y flexibilidad
- Tenés históricos crudos para auditar, reentrenar modelos o cambiar lógica
- Separás responsabilidades: los ingenieros cargan, los analistas transforman


## On-prem vs. cloud


### On-premise
- Ventajas:
  - Control total de hardware y seguridad
  - Cumplimiento de regulaciones estrictas (por ej., en bancos o gobiernos)
  - Bajos costos si ya hay infraestructura existente
- Desventajas:
  - Alto costo inicial de instalación y mantenimiento
  - Escalabilidad limitada
  - Requiere personal técnico in-house

- Cuándo usarlo:
  - Si se necesita cumplir normas muy estrictas de residencia de datos
  - Si ya se cuenta con un datacenter corporativo y equipo especializado

### Cloud
- Ventajas:
  - Escalable y elástico (pagás por lo que usás)
  - Bajo tiempo de implementación
  - Acceso desde cualquier lugar
  - Ideal para modelos ágiles y experimentación
- Desventajas:
  - Costos variables si no se gestionan bien
  - Dependencia del proveedor
  - Puede haber riesgos de compliance si no se configura correctamente
- Cuándo usarlo:
  - Para proyectos que necesitan escalar rápidamente
  - Para analytics, IA, ciencia de datos o cargas temporales
  - Cuando no se cuenta con infraestructura propia robusta


## Teradata: qué es y para qué sirve


Teradata es una plataforma de análisis de datos, reconocida por su capacidad de procesar grandes volúmenes de información estructurada de forma rápida, robusta y escalable.

Las siguientes características destacan Teradata:
- Procesamiento Masivamente Paralelo (MPP)
  - Divide las tareas en múltiples nodos para ejecutar operaciones complejas rápidamente.
  - Ideal para grandes volúmenes de datos.
- Alta escalabilidad
  - Puede manejar desde GB hasta petabytes sin cambiar de motor.
- Enfocado en análisis, no solo almacenamiento
  - Optimizaciones avanzadas para consultas analíticas (OLAP), no solo transaccionales (OLTP).
- Amplia integración
  - Compatible con herramientas de BI (Power BI, Tableau), lenguajes (SQL, Python, R) y entornos cloud (AWS, Azure, GCP).

*Importante*: Teradata no reemplaza un data lake, dado que es más bien un data warehouse moderno. Es el lugar donde se trabaja con datos estructurados y listos para análisis o decisiones de negocio.


## Ejercicios


In [83]:
import pandas as pd
import teradatasql
import requests

In [84]:
import warnings
warnings.filterwarnings('ignore')

- teradatasql: es el cliente oficial de Teradata basado en ODBC, puro Python, soportado por Teradata. Ideal para conectar desde cualquier notebook.
- teradataml: cliente más orientado a data science y análisis con Pandas-like syntax, forma parte de ClearScape. Puede usarse junto con teradatasql para análisis más ricos.
- sqlalchemy-teradata: integra Teradata con SQLAlchemy. Permite usar pandas.read_sql() o df.to_sql() con una interfaz común. No siempre está actualizado.
- pyodbc: cliente genérico ODBC que también puede conectarse a Teradata, aunque menos recomendado que teradatasql por ser más pesado y menos específico.

Teradata, a diferencia de MySQL o PostgreSQL, no tiene una GUI gráfica propia embebida en Clearscape. Si alguna vez querés una UI como SQL Server Management Studio o pgAdmin, podés descargar Teradata Studio, un cliente oficial.

In [85]:
# Coordenadas de algunas ciudades argentinas
ciudades = {
    "Buenos Aires": (-34.61, -58.38),
    "Córdoba": (-31.42, -64.18),
    "Mendoza": (-32.89, -68.84),
    "Rosario": (-32.95, -60.66),
    "Salta": (-24.78, -65.41)
}

In [86]:
datos_clima_lista = []

# Recorremos cada ciudad y consultamos la API
for ciudad, (lat, lon) in ciudades.items():
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": "true"
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        datos_clima = response.json()["current_weather"]
        #datos_clima["time"] = pd.to_datetime(datos_clima["time"])
        datos_clima["ciudad"] = ciudad
        datos_clima_lista.append(datos_clima)
    else:
        print(f"Error al obtener datos para {ciudad}: {response.status_code}")


In [87]:
clima_df = pd.DataFrame(datos_clima_lista)
clima_df

,time,interval,temperature,windspeed,winddirection,is_day,weathercode,ciudad
0,2025-04-23T14:45,900,20.5,6.9,351,1,3,Buenos Aires
1,2025-04-23T14:45,900,21.5,4.2,121,1,3,Córdoba
2,2025-04-23T14:45,900,21.5,9.6,110,1,0,Mendoza
3,2025-04-23T14:45,900,22.3,6.6,311,1,3,Rosario
4,2025-04-23T14:45,900,19.4,5.8,68,1,3,Salta


A continuación, establecemos la conexión con una instancia demo de Teradata

In [88]:
conn = teradatasql.connect(
    host='bapro-clases-h7fjrvs721akksms.env.clearscape.teradata.com',
    user='demo_user',
    password='b4.b4Pr0!'
)

In [89]:
cursor = conn.cursor()

# Mostrar tablas del esquema demo_user
cursor.execute("""
    SELECT TableName 
    FROM DBC.TablesV 
    WHERE DatabaseName = 'demo_user'
""")

for row in cursor.fetchall():
    print(row)

['space_report']
['remove_data']
['get_data']
['ClimaActual']


Todo lo que se vio de manipulación de strings se puede usar acá
- Interpolación
- Generar las queries dinámicamente
- Iterar queries con distintas fechas
- Etc

In [90]:
# Intenta eliminar la tabla si existe
tabla_a_trabajar = 'ClimaActual'

try:
    cursor.execute(f"DROP TABLE {tabla_a_trabajar}")
except teradatasql.OperationalError as e:
    if "3807" in str(e):
        print("La tabla no existe")
    else:
        raise e


In [91]:
cursor.execute("""
    CREATE MULTISET TABLE ClimaActual (
        ciudad VARCHAR(50),
        temperature FLOAT,
        windspeed FLOAT,
        winddirection FLOAT,
        weathercode INTEGER,
        fecha_hora TIMESTAMP(0)
    );
""")


TeradataCursor uRowsHandle=51 bClosed=False

In [92]:
for _, row in clima_df.iterrows():
    cursor.execute("""
        INSERT INTO ClimaActual 
        (ciudad, temperature, windspeed, winddirection, weathercode, fecha_hora)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (
        row["ciudad"], 
        row["temperature"], 
        row["windspeed"], 
        row["winddirection"], 
        row["weathercode"], 
        row["time"]
    ))


OperationalError: [Version 20.0.0.28] [Session 1262] [Teradata Database] [Error 6760] Invalid timestamp.
 at gosqldriver/teradatasql.formatError ErrorUtil.go:85
 at gosqldriver/teradatasql.(*teradataConnection).formatDatabaseError ErrorUtil.go:223
 at gosqldriver/teradatasql.(*teradataConnection).makeChainedDatabaseError ErrorUtil.go:239
 at gosqldriver/teradatasql.(*teradataConnection).processErrorParcel TeradataConnection.go:814
 at gosqldriver/teradatasql.(*TeradataRows).processResponseBundle TeradataRows.go:2412
 at gosqldriver/teradatasql.(*TeradataRows).executeSQLRequest TeradataRows.go:898
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:736
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryContext TeradataConnection.go:1333
 at database/sql.ctxDriverQuery ctxutil.go:48
 at database/sql.(*DB).queryDC.func1 sql.go:1786
 at database/sql.withLock sql.go:3574
 at database/sql.(*DB)

In [93]:
clima_df.dtypes

time              object
interval           int64
temperature      float64
windspeed        float64
winddirection      int64
is_day             int64
weathercode        int64
ciudad            object
dtype: object

In [94]:
clima_df["time"] = pd.to_datetime(datos_clima["time"])

In [95]:
for _, row in clima_df.iterrows():
    cursor.execute("""
        INSERT INTO ClimaActual 
        (ciudad, temperature, windspeed, winddirection, weathercode, fecha_hora)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (
        row["ciudad"], 
        row["temperature"], 
        row["windspeed"], 
        row["winddirection"], 
        row["weathercode"], 
        row["time"]
    ))


Podemos crear la query dinámicamente, también:

In [96]:
campos_db = ['ciudad', 'temperature', 'windspeed', 'winddirection', 'weathercode', 'fecha_hora']

query_a_definir = f"""
INSERT INTO {tabla_a_trabajar}
({', '.join(campos_db)})
VALUES (?, ?, ?, ?, ?, ?)
"""

print(query_a_definir)


INSERT INTO ClimaActual
(ciudad, temperature, windspeed, winddirection, weathercode, fecha_hora)
VALUES (?, ?, ?, ?, ?, ?)



Ahora vamos a modificar valores de la tabla en Teradata desde este cliente.

In [97]:
cursor.execute("""
    UPDATE ClimaActual
    SET temperature = temperature + 5
    WHERE ciudad = 'Córdoba'
""")

TeradataCursor uRowsHandle=58 bClosed=False

In [98]:
cursor.execute("""
    UPDATE ClimaActual
    SET windspeed = 80
    WHERE ciudad = 'Salta'
""")

TeradataCursor uRowsHandle=59 bClosed=False

Ahora cargamos la tabla desde Teradata hacia un DataFrame en Pandas

In [99]:
# Importante: hay que tener sqlalchemy actualizado para poder hacer el siguiente proceso.
# Correr en la consola lo siguiente: pip install --upgrade sqlalchemy

clima_actualizado_df = pd.read_sql("SELECT * FROM ClimaActual", conn)
clima_actualizado_df

,ciudad,temperature,windspeed,winddirection,weathercode,fecha_hora
0,Córdoba,26.5,4.2,121.0,3,2025-04-23 14:45:00
1,Buenos Aires,20.5,6.9,351.0,3,2025-04-23 14:45:00
2,Rosario,22.3,6.6,311.0,3,2025-04-23 14:45:00
3,Mendoza,21.5,9.6,110.0,0,2025-04-23 14:45:00
4,Salta,19.4,80.0,68.0,3,2025-04-23 14:45:00


In [100]:
# Antes:
clima_df

,time,interval,temperature,windspeed,winddirection,is_day,weathercode,ciudad
0,2025-04-23 14:45:00,900,20.5,6.9,351,1,3,Buenos Aires
1,2025-04-23 14:45:00,900,21.5,4.2,121,1,3,Córdoba
2,2025-04-23 14:45:00,900,21.5,9.6,110,1,0,Mendoza
3,2025-04-23 14:45:00,900,22.3,6.6,311,1,3,Rosario
4,2025-04-23 14:45:00,900,19.4,5.8,68,1,3,Salta


In [101]:
# Después
clima_actualizado_df

,ciudad,temperature,windspeed,winddirection,weathercode,fecha_hora
0,Córdoba,26.5,4.2,121.0,3,2025-04-23 14:45:00
1,Buenos Aires,20.5,6.9,351.0,3,2025-04-23 14:45:00
2,Rosario,22.3,6.6,311.0,3,2025-04-23 14:45:00
3,Mendoza,21.5,9.6,110.0,0,2025-04-23 14:45:00
4,Salta,19.4,80.0,68.0,3,2025-04-23 14:45:00
